In [1]:
# Установка Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Запуск Ollama сервера в фоновом режиме
!nohup ollama serve &

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'


In [2]:
!pip install ollama

In [3]:
import ollama
import time

time.sleep(5)

print("Загружаем первую модель...")
!ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf

print("Загружаем вторую модель...")
!ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF

Загружаем первую модель...

Загружаем вторую модель...



In [4]:
!ollama list

NAME                                                 ID              SIZE      MODIFIED               
hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:latest    042cf58aa32f    807 MB    Less than a second ago    
hf.co/CompendiumLabs/bge-base-en-v1.5-gguf:latest    98c4eb4a3287    68 MB     13 seconds ago            


In [6]:
!wget https://huggingface.co/ngxson/demo_simple_rag_py/raw/main/cat-facts.txt

!head -n 10 cat-facts.txt

--2025-10-27 12:26:30--  https://huggingface.co/ngxson/demo_simple_rag_py/raw/main/cat-facts.txt
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.17, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22657 (22K) [text/plain]
Saving to: ‘cat-facts.txt’

cat-facts.txt       100%[===================>]  22.13K  --.-KB/s    in 0.001s  

2025-10-27 12:26:30 (24.5 MB/s) - ‘cat-facts.txt’ saved [22657/22657]

On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.
Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.
When a cat chases its prey, it keeps its head level. Dogs and humans bob their heads up and down.
The technical term for a cat’s hairball is a “bezoar.”
A group of cats is called a “clowder.”
Female cats tend to be right pawe

In [7]:
dataset = []
with open('cat-facts.txt', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')


Loaded 150 entries


In [8]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))


In [9]:
for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to the database')


Added chunk 1/150 to the database
Added chunk 2/150 to the database
Added chunk 3/150 to the database
Added chunk 4/150 to the database
Added chunk 5/150 to the database
Added chunk 6/150 to the database
Added chunk 7/150 to the database
Added chunk 8/150 to the database
Added chunk 9/150 to the database
Added chunk 10/150 to the database
Added chunk 11/150 to the database
Added chunk 12/150 to the database
Added chunk 13/150 to the database
Added chunk 14/150 to the database
Added chunk 15/150 to the database
Added chunk 16/150 to the database
Added chunk 17/150 to the database
Added chunk 18/150 to the database
Added chunk 19/150 to the database
Added chunk 20/150 to the database
Added chunk 21/150 to the database
Added chunk 22/150 to the database
Added chunk 23/150 to the database
Added chunk 24/150 to the database
Added chunk 25/150 to the database
Added chunk 26/150 to the database
Added chunk 27/150 to the database
Added chunk 28/150 to the database
Added chunk 29/150 to the dat

In [10]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)


In [11]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]


In [12]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
'''


Ask me a question: how fast can cats run
Retrieved knowledge:
 - (similarity: 0.79) A cat can travel at a top speed of approximately 31 mph (49 km) over a short distance.

 - (similarity: 0.71) A cat’s heart beats nearly twice as fast as a human heart, at 110 to 140 beats a minute.

 - (similarity: 0.68) Some cats have survived falls of over 65 feet (20 meters), due largely to their “righting reflex.” The eyes and balance organs in the inner ear tell it where it is in space so the cat can land on its feet. Even cats without a tail have this ability.



In [13]:
stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)


Chatbot response:
A cat's top speed over a short distance is approximately 31 mph (49 km).